In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.nasnet import NASNetMobile
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
import numpy as np

In [ ]:
!pip install tensorflow==2.2.0rc2

In [2]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc2


In [2]:
num_class = 5
def creat_compiled_model():
    model = Sequential()
    model.add(NASNetMobile(input_shape=(224, 224, 3), include_top=False,
                           weights='imagenet', pooling='avg'))
    model.add(Dense(num_class, activation='softmax'))
    model.layers[0].trainable = True
    model.layers[1].trainable = True
    sgd = SGD(learning_rate=0.01, momentum=0.01, nesterov=False)
    model.compile(optimizer=sgd, loss='hinge',  metrics=['accuracy'])
    return model

model2 = creat_compiled_model()
model3 = creat_compiled_model()

In [3]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
# https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit_generator
train_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory('labelled_data', batch_size=4)
model3.fit(x=train_generator,epochs=2, steps_per_epoch=2 )

Found 99 images belonging to 5 classes.
Epoch 1/2
2/2 [==============================] - 1s 582ms/step - loss: 1.1405 - accuracy: 0.1250
Epoch 2/2
2/2 [==============================] - 1s 554ms/step - loss: 1.1212 - accuracy: 0.1250


In [7]:
pred_labels, pred_scores = [], []
list_of_labels = ["Back", "Discard", "Front", "Left", "Right"]
image = load_img('./labelled_data/Front/00036.jpg', target_size=(224, 224))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# cette ligne a verifier le self?
probs = model3.predict(image)[0]
print(probs)
label_idx = np.argmax(probs)
label = list_of_labels[label_idx]
score = probs[label_idx]
pred_labels.append(label)
pred_scores.append(score.item())
print(pred_labels)
print(pred_scores)

[1.2880607e-06 1.8086722e-03 1.3093783e-06 9.9807203e-01 1.1667530e-04]
['Left']
[0.9980720281600952]


In [ ]:
model3.layers[2].trainable = False

In [11]:
import os 
path = os.path.join("labelled_data", "trained_model")
#model3.layers[0].trainable = False
#model3.layers[1].trainable = False
model3.save(path,save_format="tf")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: labelled_data/trained_model/assets


In [12]:
import os 
path = os.path.join("labelled_data", "trained_model")
loaded_model = load_model(path)

In [13]:
pred_labels, pred_scores = [], []
list_of_labels = ["Back", "Discard", "Front", "Left", "Right"]
image = load_img('./labelled_data/Front/00036.jpg', target_size=(224, 224))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# cette ligne a verifier le self?
probs = loaded_model.predict(image)[0]
print(probs)
label_idx = np.argmax(probs)
label = list_of_labels[label_idx]
score = probs[label_idx]
pred_labels.append(label)
pred_scores.append(score.item())
print(pred_labels)
print(pred_scores)

[1.2880607e-06 1.8086722e-03 1.3093783e-06 9.9807203e-01 1.1667530e-04]
['Left']
[0.9980720281600952]


In [11]:
model3.summary()
mm = model3.layers[0]

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
NASNet (Model)               (None, 1056)              4269716   
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 5285      
Total params: 4,275,001
Trainable params: 4,238,263
Non-trainable params: 36,738
_________________________________________________________________


In [6]:
import os 
path = os.path.join("labelled_data", "model")
model3.save(path, overwrite=True, include_optimizer=False,save_format='h5')

https://www.tensorflow.org/api_docs/python/tf/keras/Model#save

In [8]:
mod = load_model("labelled_data/model",compile=False)
#mod.summary()


In [63]:
pred_labels, pred_scores = [], []
list_of_labels = ["Back", "Discard", "Front", "Left", "Right"]
image = load_img('./labelled_data/Front/00044.jpg', target_size=(224, 224))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# cette ligne a verifier le self?
probs = mod.predict(image)[0]
print(probs)
label_idx = np.argmax(probs)
label = list_of_labels[label_idx]
score = probs[label_idx]
pred_labels.append(label)
pred_scores.append(score.item())
print(pred_labels)
print(pred_scores)

[3.0930175e-06 7.0649298e-07 1.0134218e-07 9.9999607e-01 3.4672703e-12]
['Left']
[0.9999960660934448]


In [64]:
pred_labels, pred_scores = [], []
list_of_labels = ["Back", "Discard", "Front", "Left", "Right"]
image = load_img('./labelled_data/Front/00044.jpg', target_size=(224, 224))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# cette ligne a verifier le self?
probs = model3.predict(image)[0]
print(probs)
label_idx = np.argmax(probs)
label = list_of_labels[label_idx]
score = probs[label_idx]
pred_labels.append(label)
pred_scores.append(score.item())
print(pred_labels)
print(pred_scores)

[3.0930175e-06 7.0649298e-07 1.0134218e-07 9.9999607e-01 3.4672703e-12]
['Left']
[0.9999960660934448]


In [29]:
print(model3.optimizer.get_config())


{'name': 'SGD', 'learning_rate': 0.01, 'decay': 0.0, 'momentum': 0.01, 'nesterov': False}


NameError: name 'loss' is not defined

In [26]:
print(mod.optimizer.get_config() == model3.optimizer.get_config())
print(mod.optimizer.get_gradients() == model3.optimizer.get_gradients())

True


TypeError: get_gradients() missing 2 required positional arguments: 'loss' and 'params'

In [ ]:
tf.saved_model.save(model3,"labelled_data")

In [ ]:
loaded = tf.saved_model.load("labelled_data")
modd = build_model(loaded)


In [ ]:
pred_labels, pred_scores = [], []
list_of_labels = ["Back", "Discard", "Front", "Left", "Right"]
image = load_img('./labelled_data/Front/00036.jpg', target_size=(224, 224))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# cette ligne a verifier le self?
probs = modd.predict(image)[0]
print(probs)
label_idx = np.argmax(probs)
label = list_of_labels[label_idx]
score = probs[label_idx]
pred_labels.append(label)
pred_scores.append(score.item())
print(pred_labels)
print(pred_scores)

In [ ]:
loaded_model = load_model(path)

In [ ]:
pred_labels, pred_scores = [], []
list_of_labels = ["Back", "Discard", "Front", "Left", "Right"]
image = load_img('./labelled_data/Front/00036.jpg', target_size=(224, 224))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# cette ligne a verifier le self?
probs = loaded_model.predict(image)[0]
print(probs)
label_idx = np.argmax(probs)
label = list_of_labels[label_idx]
score = probs[label_idx]
pred_labels.append(label)
pred_scores.append(score.item())
print(pred_labels)
print(pred_scores)

In [ ]:
loaded_model = load_model(path)

In [ ]:
loaded_model = load_model(path, compile=False)
pred_labels, pred_scores = [], []
list_of_labels = ["Back", "Discard", "Front", "Left", "Right"]
image = load_img('/c/Users/yfakihan/workspace/POC/POC1/car_dataset/cars_test/00025.jpg', target_size=(224, 224))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# cette ligne a verifier le self?
probs = loaded_model.predict(image)[0]
print(probs)
label_idx = np.argmax(probs)
label = list_of_labels[label_idx]
score = probs[label_idx]
pred_labels.append(label)
pred_scores.append(score.item())
print(pred_labels)
print(pred_scores)

In [ ]:
print(loaded_model)

----------------------
----------------------

In [1]:
import logging
from logging.handlers import RotatingFileHandler

In [2]:
# création de l'objet logger qui va nous servir à écrire dans les logs
logger = logging.getLogger()
# on met le niveau du logger à DEBUG, comme ça il écrit tout
logger.setLevel(logging.DEBUG)


# création d'un formateur qui va ajouter le temps, le niveau
# de chaque message quand on écrira un message dans le log
formatter = logging.Formatter('%(asctime)s :: %(levelname)s :: %(message)s')
# création d'un handler qui va rediriger une écriture du log vers
# un fichier en mode 'append', avec 1 backup et une taille max de 1Mo
file_handler = RotatingFileHandler('labelled_data/activity.log', 'a', 1000000, 1)
# on lui met le niveau sur DEBUG, on lui dit qu'il doit utiliser le formateur
# créé précédement et on ajoute ce handler au logger
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)
# création d'un second handler qui va rediriger chaque écriture de log
# sur la console
stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.DEBUG)
logger.addHandler(stream_handler)

In [3]:
# Après 3 heures, on peut enfin logguer
# Il est temps de spammer votre code avec des logs partout :
#logger.info('Hello')
#logger.warning('Testing %s', 'foo')
logger.debug("this is a fucking message !")


this is a fucking message !


In [4]:
logger.debug("I am coming home, coming home, all ....")
logger.error("Le nombre de sous dossier est differents du nombre de classe ")

I am coming home, coming home, all ....
Le nombre de sous dossier est differents du nombre de classe 


In [5]:
import os

number_of_folders = 0

for _, dirnames, x in os.walk("./labelled_data/"):
    number_of_folders += len(dirnames)
print(f"the number of folders is : {number_of_folders}")

the number of folders is : 5


In [6]:
open("./labe")

FileNotFoundError: [Errno 2] No such file or directory: './labe'

In [7]:
open("imaginary.txt")

FileNotFoundError: [Errno 2] No such file or directory: 'imaginary.txt'

In [21]:
number_of_folders = 4
if number_of_folders != 5:
    raise FileNotFoundError ('The number of sub directories is differents from the number of classes ')

FileNotFoundError: The number of sub directories is differents from the number of classes 